In [1]:
from funct import *

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from langchain_community.utilities import SQLDatabase

# 🔹 Step 1: Connect to SQLite Database
db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite")

# 🔹 Step 2: Run the Query to Get Words
query = "SELECT word FROM words WHERE fluency = 5;"
result = db.run(query)

# 🔹 Step 3: Print Results
print("Words with fluency = 5:", result)


Words with fluency = 5: [(' tout',), (' chat',), (' silence',), (' fruit',)]


In [3]:
import ollama

client = ollama.Client()

model="gemma3:4b" 

def sentence_gen(word):
    '''creates a simple example sentnece using the word provided as well as words from ones libary based on your fluceny level of them
    - sort by word only above fluency level 3.    or something. 
    '''

    # frist attempt is basic and will just return any sentence with word in it.
    prompt = f"Only return a very basic example of a French sentence that includes this french word: '{word}'. if possible try to use a word from this list that follows but ONLY if it makes sense else do not use any words in list: {result}"

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]


def create_example():
    '''Creates a setnence with knowelge of previous messages for the user to decode the message will take in info for words knowen'''

    prompt = f"Generate an easy french sentence using words from the list attached, try to be creative while also being easy to understand. Return only that french sentence{result}"

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]  # needs some level of randomization


def test_translation(french, response):

    prompt = f"Is this: {response}. a good english translation of this french sentence {french}? Return *only* 1 sentence saying Correct/A bit off/Incorrect if wrong what is wrong."

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]  # needs some level of randomization


french_sentence = create_example()
print(french_sentence)
output = test_translation(french_sentence, "It's about a cat eating fruit in silence?")
print(output)


Le chat aime le fruit dans le silence.
A bit off. 

The French sentence emphasizes the *cat's* enjoyment of the fruit, while "It's about a cat eating fruit in silence" focuses more on the action and the quietness surrounding it.


In [4]:
#exiting fluceny here
from langchain_community.utilities import SQLDatabase

# 🔹 Step 1: Connect to SQLite Database
db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite")

# 🔹 Step 2: Define the Update Query
update_query = "UPDATE words SET fluency = 5 WHERE id = 177;"

# 🔹 Step 3: Execute the Query
db.run(update_query)

print("Fluency updated for ID.")


Fluency updated for ID.


In [5]:


# print(sentence_gen("et"))
# print(sentence_gen("cause"))
# print(sentence_gen("chat"))
# print(sentence_gen("papa"))
# print(sentence_gen("que"))
# print(sentence_gen("souvenir"))



In [6]:
def give_me_a_word():
    '''Will open up a window and will give the next practice word for you to learn or to review pygame pop up? -- should call from most and 
    using the gen sentece pornounce and more
    '''

    return 0


def pick_next_word():
    '''will use an algo to start picking next words ect..'''

    return 0

# this needs to be in this directly else it won't work on newly added mp3
def mp3_play(word):
    mp3_path = f"Pronunciations\{word}.mp3"

    if not os.path.exists(mp3_path):  
       print("MP3 file not found.") 
       return
    
    pygame.mixer.init()
    pygame.mixer.music.load(f"Pronunciations\{word}.mp3")
    pygame.mixer.music.play()
    
    print(f"Playing: {word}")

    # this might be required later so i'll keep it open like this
    while pygame.mixer.music.get_busy():  # Keep script running while music plays
        continue
    else:
        pygame.mixer.quit()  # closes it so less of a problem for other functions


In [25]:
import pygame

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 500, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pygame Lang-app V0.2")

# Colors
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
DARK_GRAY = (150, 150, 150)
LIGHT_GRAY = (200, 200, 200)
BLUE = (0, 122, 204)
BLACK = (0, 0, 0)

# Font
font = pygame.font.Font(None, 50)
button_font = pygame.font.Font(None, 30)

# Game state
current_screen = "main"

#for phrase input section
input_box = pygame.Rect(50, 300, 350, 50)
input_text = ""
active = False

button_checker = pygame.Rect(50, 375, 125, 50)
button_text_checker = "Check"

result_text = '--'


 
# Button properties
button1_rect = pygame.Rect(100, 350, 300, 50)
button2_rect = pygame.Rect(100, 420, 300, 50)
button3_rect = pygame.Rect(100, 490, 300, 50)
function_button_rect = pygame.Rect(50, 200, 120, 50)
print_button_rect = pygame.Rect(50, 500, 120, 50)
next_w_button_rect = pygame.Rect(300, 500, 120, 50)
p_button_word_react = pygame.Rect(50, 270, 120, 50)
p_button_sent_react = pygame.Rect(300, 270, 120, 50)



def draw_main_screen():
    screen.fill(WHITE)
    title_text = font.render("Mercurius Demo ", True, BLACK)
    screen.blit(title_text, (110, 100))
    
    pygame.draw.rect(screen, BLUE, button1_rect)
    pygame.draw.rect(screen, BLUE, button2_rect)
    pygame.draw.rect(screen, BLUE, button3_rect)
    
    screen.blit(button_font.render("Study", True, WHITE), button1_rect.move(50, 15))
    screen.blit(button_font.render("DO **NOT** CLICK HERE", True, WHITE), button2_rect.move(20, 15))
    screen.blit(button_font.render("Complete Phrases", True, WHITE), button3_rect.move(50, 15))

word_index = 0
val = 0 
current_phrase = 'base phrase, with some words but not that many'

def get_current_word():
    #global word_index
    return words[word_index]

function_output = ""

def function_action():
    global function_output
    function_output = sentence_gen(words[word_index].lower())

def card_display():
    word = get_current_word()
    global val
    if val == 0:
        return word
    else: 
        return word + " | " + generate_def(word)

def next_word():
    global word_index
    global function_output
    word_index = (word_index + 1) % len(words)
    function_output = ""

def gen_phrase():
    global current_phrase
    current_phrase = create_example()

def wrap_text(text, font, max_width):
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + " " + word if current_line else word
        test_surface = font.render(test_line, True, BLACK)
        if test_surface.get_width() <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word  # Start a new line with the current word
    
    if current_line:  # Append the last line
        lines.append(current_line)
    
    return lines


def draw_phrases_screen():
    screen.fill(WHITE)
    word_box_rect = pygame.Rect(50, 75, 400, 200)
    pygame.draw.rect(screen, DARK_GRAY, word_box_rect, border_radius=10)

    max_width = word_box_rect.width - 20  # Padding inside the box
    y_offset = word_box_rect.y + 10  # Start inside the box with padding

    lines = wrap_text(current_phrase, font, max_width)

    # Render and blit each wrapped line
    for line in lines:
        text_surface = font.render(line, True, BLACK)
        screen.blit(text_surface, (word_box_rect.x + 10, y_offset))
        y_offset += font.get_height()  # Move down for the next line

        # Draw input box
    pygame.draw.rect(screen, LIGHT_GRAY if active else DARK_GRAY, input_box, border_radius=5)
    input_surface = font.render(input_text, True, BLACK)
    screen.blit(input_surface, (input_box.x + 10, input_box.y + 10))

    # Draw button
    pygame.draw.rect(screen, BLUE, button_checker, border_radius=5)
    button_surface = font.render(button_text_checker, True, WHITE)
    screen.blit(button_surface, (button_checker.x + 10, button_checker.y + 10))

    # Draw result text
    #result_surface = font.render(result_text, True, BLACK)
    #screen.blit(result_surface, (50, 450))

    lines_2 = wrap_text(result_text, font, 400)
    y_offset = 450
    # Render and blit each wrapped line
    for line in lines_2:
        result_surface = font.render(line, True, BLACK)
        screen.blit(result_surface, (50, y_offset))
        y_offset += 35  # Move down for the next line

    



def draw_review_screen():

    screen.fill(WHITE)
    word_box_rect = pygame.Rect(100, 50, 300, 60)
    pygame.draw.rect(screen, DARK_GRAY, word_box_rect, border_radius=10)


  # Draw the word in the center of the box
    text_surface = font.render(card_display(), True, BLACK)
    text_rect = text_surface.get_rect(center=word_box_rect.center)
    screen.blit(text_surface, text_rect)

    
    
    pygame.draw.rect(screen, BLUE, function_button_rect)
    pygame.draw.rect(screen, BLUE, print_button_rect)
    pygame.draw.rect(screen, BLUE, next_w_button_rect)
    pygame.draw.rect(screen, BLUE, p_button_word_react)
    pygame.draw.rect(screen, BLUE, p_button_sent_react)
    
    screen.blit(button_font.render("Example", True, WHITE), function_button_rect.move(20, 15))
    screen.blit(button_font.render("English", True, WHITE), print_button_rect.move(35, 15))
    screen.blit(button_font.render("Next", True, WHITE), next_w_button_rect.move(35, 15))
    screen.blit(button_font.render("P_Word", True, WHITE), p_button_word_react.move(20, 15))
    screen.blit(button_font.render("P_Sentence", True, WHITE), p_button_sent_react.move(20, 15))

    output_surface = button_font.render((function_output.split(".")[0]).split("?")[0], True, BLACK)
    screen.blit(output_surface, (180, 215))




def main():
    global current_screen
    global val
    global result_text
    global input_text
    running = True
    
    while running:
        screen.fill(WHITE)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if current_screen == "main":
                    if button1_rect.collidepoint(event.pos):
                        current_screen = "review"
                    elif button2_rect.collidepoint(event.pos): # the "Do not press button"
                        pass  
                    elif button3_rect.collidepoint(event.pos):
                        gen_phrase()
                        current_screen = "phrases"
                if current_screen == "review":
                    if function_button_rect.collidepoint(event.pos):
                        function_action()
                    elif p_button_word_react.collidepoint(event.pos):
                        mp3_play(get_current_word())
                    elif p_button_sent_react.collidepoint(event.pos):
                        create_pronunciation(function_output.split(".")[0])
                        mp3_play(function_output.split(".")[0])
                    elif print_button_rect.collidepoint(event.pos):
                        val = 1
                    elif next_w_button_rect.collidepoint(event.pos):
                        val = 0
                        next_word()
                if current_screen == "phrases":
                    if input_box.collidepoint(event.pos):
                        active = True
                    else:
                        active = False

                    if button_checker.collidepoint(event.pos): 
                        result_text = test_translation(current_phrase, (input_text.lower().strip()))
                        # if input_text.lower().strip() == "please":
                        #     result_text = "You Pass-- will change to global var from func call"
                        # else:
                        #     result_text = "i guess call again but will leave to test"

            elif event.type == pygame.KEYDOWN:
                if active:
                    if event.key == pygame.K_RETURN:
                        result_text = test_translation(current_phrase, (input_text.lower().strip()))
                        #else:
                        #    result_text = "Not the magic word"
                        input_text = ""  # Clear input
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]  # Remove last character
                    else:
                        input_text += event.unicode  # Add new character
                                
                        

                        
                
        if current_screen == "main":
            draw_main_screen()
        elif current_screen == "review":
            draw_review_screen()
        elif current_screen == "phrases":
            draw_phrases_screen()
        
        pygame.display.flip()
    
    pygame.quit()

if __name__ == "__main__":
    main()

In [10]:
import pygame

pygame.init()

# Screen setup
WIDTH, HEIGHT = 500, 300
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Magic Word Checker")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
DARK_GRAY = (50, 50, 50)
LIGHT_GRAY = (200, 200, 200)
BLUE = (70, 130, 180)

# Fonts
font = pygame.font.Font(None, 30)

# Input Box
input_box = pygame.Rect(50, 100, 300, 40)
input_text = ""
active = False

# Button
button = pygame.Rect(370, 100, 100, 40)
button_text = "Submit"

# Result Text
result_text = ""

# Main loop
running = True
while running:
    screen.fill(WHITE)

    # Draw input box
    pygame.draw.rect(screen, LIGHT_GRAY if active else DARK_GRAY, input_box, border_radius=5)
    input_surface = font.render(input_text, True, BLACK)
    screen.blit(input_surface, (input_box.x + 10, input_box.y + 10))

    # Draw button
    pygame.draw.rect(screen, BLUE, button, border_radius=5)
    button_surface = font.render(button_text, True, WHITE)
    screen.blit(button_surface, (button.x + 10, button.y + 10))

    # Draw result text
    result_surface = font.render(result_text, True, BLACK)
    screen.blit(result_surface, (50, 180))

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if input_box.collidepoint(event.pos):
                active = True
            else:
                active = False

            if button.collidepoint(event.pos):  # Button clicked
                if input_text.lower().strip() == "please":
                    result_text = "You Pass"
                else:
                    result_text = "Not the magic word"

        elif event.type == pygame.KEYDOWN:
            if active:
                if event.key == pygame.K_RETURN:
                    if input_text.lower().strip() == "please":
                        result_text = "You Pass"
                    else:
                        result_text = "Not the magic word"
                    input_text = ""  # Clear input
                elif event.key == pygame.K_BACKSPACE:
                    input_text = input_text[:-1]  # Remove last character
                else:
                    input_text += event.unicode  # Add new character

    pygame.display.flip()

pygame.quit()
